In [1]:
import pandas as pd
import numpy as np
import gmaps
import requests
import os
import time
import json
import matplotlib.pyplot as plt
from citipy import citipy
from datetime import datetime
from config import g_key
from config import weather_api_key
from scipy.stats import linregress

In [2]:
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
coordinates = list(lat_lngs)

In [4]:
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

len(cities)

738

In [5]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [6]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [11]:
# Loop through all the cities in our list.
for i, city in enumerate(cities):
    # print(f'I:{i}, City:{city}')
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        
    # Create endpoint URL with each city.
    city_url = f'{url}&q={city.replace(" ","+")}'
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        description = city_weather['weather'][0]['description']
        
        # Convert the date to ISO
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information to the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skip

In [12]:
print(f'Len of city data: {len(city_data)}')

Len of city data: 1358


In [13]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,Vila Do Maio,CV,15.1333,-23.2167,70.59,73,59,11.83,broken clouds,0.0,0.0
1,Ushuaia,AR,-54.8000,-68.3000,35.26,80,100,18.41,sleet,0.0,0.0
2,Manokwari,ID,-0.8667,134.0833,83.08,75,99,9.78,overcast clouds,0.0,0.0
3,Rikitea,PF,-23.1203,-134.9692,78.51,64,51,12.50,broken clouds,0.0,0.0
4,Dicabisagan,PH,17.0818,122.4157,77.31,88,100,10.92,overcast clouds,0.0,0.0
5,Tahta,EG,26.7693,31.5021,75.70,13,0,3.98,clear sky,0.0,0.0
6,New Norfolk,AU,-42.7826,147.0587,58.50,71,100,3.00,overcast clouds,0.0,0.0
7,Leningradskiy,RU,69.3833,178.4167,-1.70,86,48,4.83,scattered clouds,0.0,0.0
8,Hilo,US,19.7297,-155.0900,76.73,82,75,9.22,moderate rain,0.0,0.0
9,Atasu,KZ,48.6814,71.6433,47.19,67,26,9.06,scattered clouds,0.0,0.0


In [15]:
# Export the DataFrame as a CSV file, and save it as WeatherPy_Database.csv in the Weather_Database folder
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")